# **D1209_work_김현우**

- 피쳐 컬럼 타입에 따른 전처리
    - 범주형 피쳐 : 명목형/순서형 = > 인코딩
        - -> 명목형 => OneHotEncoding
        - -> 순서형 => OrdinalEncoding
    
    - 수치형 피쳐
        - -> 범주형 변환 ? : 데이터가 가진 의미 체크
        - -> 수치형 => 스케일링 여부

- 타겟 컬럼 타입에 따른 전처리
    - 범주형 타겟 -> LabelEncoding
    - 수치형 타겟 -> 그대로

- 사용 파일: titaninc.csv, auto_mpg.csv

### 1. titanic.csv

[1] 모듈 로딩 및 데이터 준비 <HR>

In [7]:
# ============================================================================
# [1-1] 모듈 로딩
# ============================================================================
# 제작 함수
import preprocessing    # 데이터 기본 전처리

# 데이터 전처리
import pandas as pd
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns

# 머신러닝 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV    # 튜닝 관련
from sklearn.neighbors import KNeighborsClassifier                      # 학습 알고리즘
from sklearn.model_selection import train_test_split                    # 데이터셋 관련
from sklearn.preprocessing import StandardScaler

In [8]:
# ============================================================================
# [1-2] 데이터 준비(titaninc.csv)
# ============================================================================
FILE_NAME1 = '../Data/titanic.csv'
titanicDF = pd.read_csv(FILE_NAME1)

In [12]:
# ============================================================================
# [1-3] 데이터 확인
# ============================================================================
preprocessing.check_data(titanicDF)
# ============================================================================
# survived      : 생존 여부 (0=사망, 1=생존)
# pclass        : 객실 등급 (1=1등석, 2=2등석, 3=3등석)
# sex           : 성별 (male=남성, female=여성)
# age           : 나이
# sibsp         : 함께 탑승한 형제자매 및 배우자 수
# parch         : 함께 탑승한 부모 및 자녀 수
# fare          : 요금 (티켓 가격)
# embarked      : 탑승 항구 이니셜 (C, Q, S)
# class         : 객실 등급 (First, Second, Third)
# who           : 구분 (man, woman, child)
# adult_male    : 성인 남성 여부 (True/False)
# deck          : 선실 번호 첫 글자 (데크 위치)
# embark_town   : 탑승 항구 이름 (Southampton 등)
# alive         : 생존 여부 (yes/no)
# alone         : 혼자 탑승 여부 (True/False)
# ============================================================================

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


None

[2] 피쳐와 타겟 분리 <hr>

In [19]:
# ============================================================================
# 1. 타겟
# ============================================================================

# survived: (0 또는 1)

# 추가
# alive는 survived와 동일한 의미를 가지기 때문에 삭제하기로 했음
targetSR = titanicDF["survived"]
print(targetSR)

# ============================================================================
# 2. 피쳐
# ============================================================================

# === 선택한 피쳐 ===
# pclass        : 객실 등급
# sex           : 성별
# age           : 나이
# sibsp         : 동승한 형제/배우자 수
# parch         : 동승한 부모/자녀 수
# fare          : 요금
# embarked      : 탑승 항구
# deck          : 선실 번호

# === 버릴 피쳐 ===
# 의미가 중복되거나 조합을 해서 만든 컬럼들은 삭제하기로 함
# class         :   pclass와 의미가 겹침 (숫자형인  pclass을 사용하기로 함)
# embark_town   : embarked와 의미가 겹침 (더 짧은 embarked를 사용하기로 함)
# who           :   sex와   age를 조합
# adult_male    :   sex와   age를 조합
# alone         : sibsp와 parch를 조합

featureDF = titanicDF[["pclass", "sex", "age", "sibsp", "parch", "fare", "embarked", "deck"]]
print(featureDF)

# ============================================================================


0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: survived, Length: 891, dtype: int64
     pclass     sex   age  sibsp  parch     fare embarked deck
0         3    male  22.0      1      0   7.2500        S  NaN
1         1  female  38.0      1      0  71.2833        C    C
2         3  female  26.0      0      0   7.9250        S  NaN
3         1  female  35.0      1      0  53.1000        S    C
4         3    male  35.0      0      0   8.0500        S  NaN
..      ...     ...   ...    ...    ...      ...      ...  ...
886       2    male  27.0      0      0  13.0000        S  NaN
887       1  female  19.0      0      0  30.0000        S    B
888       3  female   NaN      1      2  23.4500        S  NaN
889       1    male  26.0      0      0  30.0000        C    C
890       3    male  32.0      0      0   7.7500        Q  NaN

[891 rows x 8 columns]


### 2. auto_mpg

[1] 모듈 로딩 및 데이터 준비 <HR>

In [10]:
# --------------------------------------------------
# [1-2] 데이터 준비(auto_mpg.csv)
# --------------------------------------------------
FILE_NAME2 = '../Data/auto_mpg.csv'
autoDF = pd.read_csv(FILE_NAME2)

In [11]:
# --------------------------------------------------
# [1-3] 데이터 확인
# --------------------------------------------------
preprocessing.check_data(autoDF)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


None